# TiDB

> [TiDB Cloud](https://tidbcloud.com/) 是一个全面的数据库即服务 (DBaaS) 解决方案，提供专用和无服务器选项。TiDB Serverless 现已将内置的向量搜索集成到 MySQL 生态系统中。通过此增强功能，您可以使用 TiDB Serverless 无缝开发 AI 应用，而无需引入新数据库或额外的技术栈。立即加入私有 Beta 等待名单，体验抢先一步：https://tidb.cloud/ai。

本笔记本将介绍如何在 langchain 中使用 `TiDBLoader` 从 TiDB 加载数据。

## 先决条件

在使用 `TiDBLoader` 之前，我们将安装以下依赖项：

In [ ]:
%pip install --upgrade --quiet langchain

接着，我们将配置与 TiDB 的连接。在本笔记本中，我们将遵循 TiDB Cloud 提供的标准连接方法，以建立安全高效的数据库连接。

In [2]:
import getpass

# copy from tidb cloud console，replace it with your own
tidb_connection_string_template = "mysql+pymysql://<USER>:<PASSWORD>@<HOST>:4000/<DB>?ssl_ca=/etc/ssl/cert.pem&ssl_verify_cert=true&ssl_verify_identity=true"
tidb_password = getpass.getpass("Input your TiDB password:")
tidb_connection_string = tidb_connection_string_template.replace(
    "<PASSWORD>", tidb_password
)

## 从 TiDB 加载数据

以下是可用于自定义 `TiDBLoader` 行为的一些关键参数：

- `query` (str): 这是将在 TiDB 数据库上执行的 SQL 查询。查询应选择您要加载到 `Document` 对象中的数据。
例如，您可以使用类似 `"SELECT * FROM my_table"` 的查询来获取 `my_table` 中的所有数据。

- `page_content_columns` (Optional[List[str]]): 指定应包含在每个 `Document` 对象的 `page_content` 中的列名列表。
如果将其设置为 `None`（默认值），则查询返回的所有列都将包含在 `page_content` 中。这允许您根据数据的特定列来定制每个文档的内容。

- `metadata_columns` (Optional[List[str]]): 指定应包含在每个 `Document` 对象的 `metadata` 中的列名列表。
默认情况下，此列表为空，这意味着除非明确指定，否则不包含任何元数据。这对于包含有关每个文档的附加信息非常有用，这些信息不构成主要内容的一部分，但对于处理或分析仍然很有价值。

In [3]:
from sqlalchemy import Column, Integer, MetaData, String, Table, create_engine

# Connect to the database
engine = create_engine(tidb_connection_string)
metadata = MetaData()
table_name = "test_tidb_loader"

# Create a table
test_table = Table(
    table_name,
    metadata,
    Column("id", Integer, primary_key=True),
    Column("name", String(255)),
    Column("description", String(255)),
)
metadata.create_all(engine)


with engine.connect() as connection:
    transaction = connection.begin()
    try:
        connection.execute(
            test_table.insert(),
            [
                {"name": "Item 1", "description": "Description of Item 1"},
                {"name": "Item 2", "description": "Description of Item 2"},
                {"name": "Item 3", "description": "Description of Item 3"},
            ],
        )
        transaction.commit()
    except:
        transaction.rollback()
        raise

In [4]:
from langchain_community.document_loaders import TiDBLoader

# Setup TiDBLoader to retrieve data
loader = TiDBLoader(
    connection_string=tidb_connection_string,
    query=f"SELECT * FROM {table_name};",
    page_content_columns=["name", "description"],
    metadata_columns=["id"],
)

# Load data
documents = loader.load()

# Display the loaded documents
for doc in documents:
    print("-" * 30)
    print(f"content: {doc.page_content}\nmetada: {doc.metadata}")

------------------------------
content: name: Item 1
description: Description of Item 1
metada: {'id': 1}
------------------------------
content: name: Item 2
description: Description of Item 2
metada: {'id': 2}
------------------------------
content: name: Item 3
description: Description of Item 3
metada: {'id': 3}


In [5]:
test_table.drop(bind=engine)